# Segementations

Notebook to split the columns (and rows) in possibly orthogonal parts using spectral clustering

## Split the collumns

First we split the matrix.
For thsi we want to group them in in two clusters of collums that are idealy orhtogonal to each other.
For this we consider a graph where the collums $c_i$ are represented by nodes.
The edges have the weight
$$ e_{i,j} = |c_i^\top c_j|$$

For this we split the graph using spectral clustering https://en.wikipedia.org/wiki/Spectral_clustering 

In [ ]:
import numpy as np
import scipy.stats 
import matplotlib.pyplot as plt

In [ ]:
N = 10
A = np.random.rand(N,N)
plt.matshow(A)

Get a tailor made example matrix

In [ ]:
k = np.ones(N,dtype=int)
k[:N//2]=0
k =np.random.permutation(k)

Q = scipy.stats.ortho_group.rvs(N)
A = Q[:,k]+1e-3*np.random.rand(N,N)
# add a small random amtrix to avoid two eigenvalues with 0 and make eigenvector unique

In [ ]:
plt.matshow(A.T@A)

In [ ]:
# Caclulate Laplacian
L = -np.abs(A.T@A) # use A@A.T if one wants to resort rows
# Set diagonal
np.fill_diagonal(L,0)
np.fill_diagonal(L,-np.sum(L,axis=0))

#normalize L
d = np.sqrt(1/np.diag(L))
L = d.reshape(1,-1)*L*d.reshape(-1,1)

In [ ]:
plt.matshow(L)

In [ ]:
w, v = np.linalg.eig(L)

We need to sort the eigenvalues and eigenvectors

In [ ]:
o = np.argsort(w.real)
v = v[:,o]
w[o]

Extract the second eigenvalue

In [ ]:
v[:,1]

Get a new permutation by ordering the elements of the eigenvalue

In [ ]:
p = np.argsort(v[:,1])

In [ ]:
v[:,1][p]

In [ ]:
#resort collumns
plt.matshow(np.abs(A[:,p].T@A[:,p]))

In [ ]:
def segment_matrix(A,normalize = True):
    """
    Matrix 
    
    returns
        s: boolena vector, True if collumn in second part 
    """
    
    # Caclulate Laplacian
    L = -np.abs(A.T@A) 
    # Set diagonal
    np.fill_diagonal(L,0)
    np.fill_diagonal(L,-np.sum(L,axis=0))

    if normalize:
        #normalize L
        d = np.sqrt(1/np.diag(L))
        L = d.reshape(1,-1)*L*d.reshape(-1,1)
        

    w, v = np.linalg.eig(L)

    #order eigenvalues
    o = np.argsort(w.real)
    v = v[:,o]
    print("Eigenvalues:",w[o])
    print("Fiedler-Vector",v[:,1])
        
    return v[:,1].real>0

In [ ]:
b = segment_matrix(A)
p = np.argsort(b)

plt.matshow(np.abs(A[:,p].T@A[:,p]))

In [ ]:
b

In [ ]:
p

In [ ]:
B = A.T

b = segment_matrix(B.T)
p = np.argsort(b)

plt.matshow(np.abs(B[p,:]@B[p,:].T))

create more advanced matrix so split horizontaly and vertically

In [ ]:
N = 6

U_a =scipy.stats.ortho_group.rvs(N)
Vt_a=scipy.stats.ortho_group.rvs(N)

U_b =scipy.stats.ortho_group.rvs(N)
Vt_b=scipy.stats.ortho_group.rvs(N)

A = np.block(
    [
        [U_a[:,:1]@Vt_a[ :1,:], U_a[:,1:2]@Vt_b[ :1,:]],
        [U_b[:,:1]@Vt_a[1:2,:], U_b[:,1:2]@Vt_b[1:2,:]]
    ]
)

plt.matshow(A)

k= np.arange(N*2)
k_col =np.random.permutation(k)
k_row =np.random.permutation(k)
ik_row =np.argsort(k_row)
ik_col =np.argsort(k_col)

A = A[k_row,:][:,k_col]
plt.matshow(A)

In [ ]:
# check if we can reverse the permutation
k_row[ik_row]

In [ ]:
s_col = segment_matrix(A)
s_row = segment_matrix(A.T)

Now check if we get beack the original segmentation: Thsi means that all the True and False values are grouped after we undo the permutation

In [ ]:
s_col[ik_col]

In [ ]:
s_row[ik_row]